# Mini-Project: Data Wrangling and Transformation with Pandas

Working with tabular data is a necessity for anyone with enterprises having a majority of their data in relational databases and flat files. This mini-project is adopted from the excellent tutorial on pandas by Brandon Rhodes which you have watched earlier in the Data Wrangling Unit. In this mini-project, we will be looking at some interesting data based on movie data from the IMDB.

This assignment should help you reinforce the concepts you learnt in the curriculum for Data Wrangling and sharpen your skills in using Pandas. Good Luck!

### Please make sure you have one of the more recent versions of Pandas

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
pd.__version__

## Taking a look at the Movies dataset
This data shows the movies based on their title and the year of release

In [ ]:
movies = pd.read_csv('titles.csv')
movies.info()

In [ ]:
movies.head()

## Taking a look at the Cast dataset

This data shows the cast (actors, actresses, supporting roles) for each movie

- The attribute `n` basically tells the importance of the cast role, lower the number, more important the role.
- Supporting cast usually don't have any value for `n`

In [ ]:
cast = pd.read_csv('cast.csv.zip')
cast.info()

In [ ]:
cast.head(10)

## Taking a look at the Release dataset

This data shows details of when each movie was release in each country with the release date

In [ ]:
release_dates = pd.read_csv('release_dates.csv', parse_dates=['date'], infer_datetime_format=True)
release_dates.info()

In [ ]:
release_dates.head()

# Section I - Basic Querying, Filtering and Transformations

### What is the total number of movies?

In [ ]:
len(movies)

### List all Batman movies ever made

In [ ]:
batman_df = movies[movies.title == 'Batman']
print('Total Batman Movies:', len(batman_df))
batman_df

### List all Batman movies ever made - the right approach

In [ ]:
batman_df = movies[movies.title.str.contains('Batman', case=False)]
print('Total Batman Movies:', len(batman_df))
batman_df.head(10)

### Display the top 15 Batman movies in the order they were released

In [ ]:
batman_df.sort_values(by=['year'], ascending=True).iloc[:15]

### Section I - Q1 : List all the 'Harry Potter' movies from the most recent to the earliest

In [ ]:
# helper method to create a boolean mask to exclude null values 
def exclude_null_feature_mask (feature, data):
    return ~data[feature].isnull() # filter out null

harry_potter_movies = movies[exclude_null_feature_mask ('title', movies) & (movies.title.str.contains('Harry potter', case=False))].sort_values(by='year', ascending=False)
print(harry_potter_movies)

### How many movies were made in the year 2017?

In [ ]:
len(movies[movies.year == 2017])

### Section I - Q2 : How many movies were made in the year 2015?

In [ ]:
len(movies[movies.year == 2015])

### Section I - Q3 : How many movies were made from 2000 till 2018?
- You can chain multiple conditions using OR (`|`) as well as AND (`&`) depending on the condition

In [ ]:
mask = (movies['year'].between(2000, 2018))
len(movies[mask])

### Section I - Q4: How many movies are titled "Hamlet"?

In [ ]:
mask = (movies['title'].str.startwith('Hamlet', case=False))
len(movies[exclude_null_feature_mask ('title', movies) & mask])

### Section I - Q5: List all movies titled "Hamlet" 
- The movies should only have been released on or after the year 2000
- Display the movies based on the year they were released (earliest to most recent)

In [ ]:
mask_year = movies['year']>=2000
mask_title = movies['title'].str.startswith('Hamlet', case=False)
mask_null = exclude_null_feature_mask ('title', movies)
movies[mask_null & mask_title & mask_year].sort_values(by='year', ascending=True)


### Section I - Q6: How many roles in the movie "Inception" are of the supporting cast (extra credits)
- supporting cast are NOT ranked by an "n" value (NaN)
- check for how to filter based on nulls

In [ ]:
cast_inception = cast['title'] == 'Inception'
len(cast[(cast_inception) & (cast['n'].isnull())])

### Section I - Q7: How many roles in the movie "Inception" are of the main cast
- main cast always have an 'n' value

In [ ]:
len(cast[(cast_inception) & (~cast['n'].isnull())])

### Section I - Q8: Show the top ten cast (actors\actresses) in the movie "Inception" 
- main cast always have an 'n' value
- remember to sort!

In [ ]:
cast[(cast_inception) & (~cast['n'].isnull())].sort_values(by='n', ascending=True).head(10)

### Section I - Q9:

(A) List all movies where there was a character 'Albus Dumbledore' 

(B) Now modify the above to show only the actors who played the character 'Albus Dumbledore'
- For Part (B) remember the same actor might play the same role in multiple movies

In [ ]:
albus_filter = cast ['character'] == 'Albus Dumbledore'
cast[albus_filter]['title']

In [ ]:
cast[albus_filter]['name'].drop_duplicates()

### Section I - Q10:

(A) How many roles has 'Keanu Reeves' played throughout his career?

(B) List the leading roles that 'Keanu Reeves' played on or after 1999 in order by year.

In [ ]:
keanu_filter = cast ['name'] == 'Keanu Reeves'
len(cast[keanu_filter])

In [ ]:
year_filter = cast['year'] >= 1999
leading_role_filter = cast['n'] == 1.0
cast[keanu_filter & year_filter & leading_role_filter].sort_values('year', ascending=True)

### Section I - Q11: 

(A) List the total number of actor and actress roles available from 1950 - 1960

(B) List the total number of actor and actress roles available from 2007 - 2017

In [ ]:
year_filter = cast['year'].between(1950, 1960)
len(cast[year_filter])

In [ ]:
year_filter2 = cast['year'].between(2007, 2017)
len(cast[year_filter2])

### Section I - Q12: 

(A) List the total number of leading roles available from 2000 to present

(B) List the total number of non-leading roles available from 2000 - present (exclude support cast)

(C) List the total number of support\extra-credit roles available from 2000 - present

In [ ]:
year_leading_roles_filter = (cast['year'] >= 2000) & (cast['n'] == 1.0) 
len(cast[year_leading_roles_filter])

In [ ]:
year_non_leading_roles_filter = (cast['year'] >= 2000) & (cast['n'] > 1.0) 
len(cast[year_non_leading_roles_filter])

In [ ]:
year_support_roles_filter = (cast['year'] >= 2000) & (cast['n'].isnull())
len(cast[year_support_roles_filter])

# Section II - Aggregations, Transformations and Visualizations

## What are the top ten most common movie names of all time?


In [ ]:
top_ten = movies.title.value_counts()[:10]
top_ten

### Plot the top ten common movie names of all time

In [ ]:
top_ten.plot(kind='barh')

### Section II - Q1:  Which years in the 2000s saw the most movies released? (Show top 3)

In [ ]:
movies[movies['year'].between(2000, 2019)].groupby('year').agg({"title":"count"}).sort_values(by='title', ascending=False).head(5)

### Section II - Q2: # Plot the total number of films released per-decade (1890, 1900, 1910,....)
- Hint: Dividing the year and multiplying with a number might give you the decade the year falls into!
- You might need to sort before plotting

In [ ]:
def get_decade(year):
    return year - (year % 100)
 
movies.sort_values(by='year', ascending=False)
movies['decade'] = movies['year'].map(get_decade)

movies_grouped_by_decade = movies.groupby('decade').agg({'title':'count'})
movies_sorted_grouped = movies_grouped_by_decade.sort_values(by='decade', ascending=True)
movies_sorted_grouped

movies_sorted_grouped.plot(kind='barh')

### Section II - Q3: 

(A) What are the top 10 most common character names in movie history?

(B) Who are the top 10 people most often credited as "Herself" in movie history?

(C) Who are the top 10 people most often credited as "Himself" in movie history?

In [ ]:
cast.groupby('character').agg({'title':'count'}).sort_values('title', ascending=False)

In [ ]:
cast[cast['character'] == 'Himself'].groupby('name').agg({'title':'count'}) .sort_values('title', ascending=False).head()

In [ ]:
cast[cast['character'] == 'Herself'].groupby('name').agg({'title':'count'}).sort_values('title', ascending=False).head()

### Section II - Q4: 

(A) What are the top 10 most frequent roles that start with the word "Zombie"?

(B) What are the top 10 most frequent roles that start with the word "Police"?

- Hint: The `startswith()` function might be useful

In [ ]:
import math 
def get_role (sr):
    if math.isnan(sr):  
       return 'Support'
    else:
        if sr == 1.0:
            return 'Main'
        else:
            return 'Second'

cast_zoombie = cast[exclude_null_feature_mask ('character', cast) & cast.character.str.startswith('Zombie')]
cast_zoombie['role'] = cast_zoombie['n'].map(get_role)

cast_zoombie.role.value_counts(ascending=False)
cast


In [ ]:
cast_police = cast[exclude_null_feature_mask ('character', cast) & cast.character.str.startswith('Police')]
cast_police['role'] = cast_police['n'].map(get_role)

cast_police.role.value_counts(ascending=False)
cast

### Section II - Q5: Plot how many roles 'Keanu Reeves' has played in each year of his career.

In [ ]:
keanu_filter = cast ['name'] == 'Keanu Reeves'
kr_over_years = cast[keanu_filter].groupby('year')['title'].agg('count')
kr_over_years.plot(kind='bar')

### Section II - Q6: Plot the cast positions (n-values) of Keanu Reeve's roles through his career over the years.


In [ ]:
keanu_filter = cast ['name'] == 'Keanu Reeves'
kr_over_years = cast[keanu_filter].groupby(['year', 'n'])['title'].agg('count')
kr_over_years.plot(kind='bar')

### Section II - Q7: Plot the number of "Hamlet" films made by each decade

In [ ]:
pd.options.mode.chained_assignment = None
hamlet_movies = movies[exclude_null_feature_mask('title', movies) & movies['title'].str.startswith('Hamlet')]
hamlet_movies['decade'] = hamlet_movies['year'].map(get_decade)

hamlet_movies_grouped_by_decade = hamlet_movies.groupby('decade').agg({'title':'count'})
hamlet_movies_grouped_by_decade.plot(kind='barh')

### Section II - Q8: 

(A) How many leading roles were available to both actors and actresses, in the 1960s (1960-1969)?

(B) How many leading roles were available to both actors and actresses, in the 2000s (2000-2009)?

- Hint: A specific value of n might indicate a leading role

In [ ]:
len(cast[(cast['n'] == 1.0) & (cast['year'].between(1960, 1969))])

In [ ]:
len(cast[(cast['n'] == 1.0) & (cast['year'].between(2000, 2009))])

### Section II - Q9: List, in order by year, each of the films in which Frank Oz has played more than 1 role.

In [ ]:
frank_oz_roles = cast[cast['name'] == 'Frank Oz'].groupby(by=['title', 'year']).agg({'title':'count'}).sort_index(level=['year'], ascending=[True])
frank_oz_roles [frank_oz_roles['title']>1]


### Section II - Q10: List each of the characters that Frank Oz has portrayed at least twice

In [ ]:
frank_oz_chars = cast[cast['name'] == 'Frank Oz'].groupby(by=['character']).agg({'character':'count'})
frank_oz_chars [frank_oz_chars['character']>1]

# Section III - Advanced Merging, Querying and Visualizations

## Make a bar plot with the following conditions
- Frequency of the number of movies with "Christmas" in their title 
- Movies should be such that they are released in the USA.
- Show the frequency plot by month

In [ ]:
christmas = release_dates[(release_dates.title.str.contains('Christmas')) & (release_dates.country == 'USA')]
christmas.date.dt.month.value_counts().sort_index().plot(kind='bar')

### Section III - Q1: Make a bar plot with the following conditions
- Frequency of the number of movies with "Summer" in their title 
- Movies should be such that they are released in the USA.
- Show the frequency plot by month

In [ ]:
summer = release_dates[(release_dates.title.str.contains('Summer')) & (release_dates.country == 'USA')]
summer.date.dt.month.value_counts().sort_index().plot(kind='bar')

### Section III - Q2: Make a bar plot with the following conditions
- Frequency of the number of movies with "Action" in their title 
- Movies should be such that they are released in the USA.
- Show the frequency plot by week

In [ ]:
release_dates
action = release_dates[(release_dates.title.str.contains('Action')) & (release_dates.country == 'USA')]
action.date.dt.day_of_week.value_counts().sort_index().plot(kind='bar')

### Section III - Q3: Show all the movies in which Keanu Reeves has played the lead role along with their   release date in the USA sorted by the date of release
- Hint: You might need to join or merge two datasets!

In [ ]:
keanu_lead_role_filter = (cast ['name'] == 'Keanu Reeves') & (cast['n'] == 1.0) 
kr_over_years = cast[keanu_lead_role_filter].merge(release_dates[release_dates['country'] == 'USA'], on=['title', 'year']).sort_values('date', ascending=False) 
kr_over_years


### Section III - Q4: Make a bar plot showing the months in which movies with Keanu Reeves tend to be released in the USA?

In [ ]:
kr_over_years.date.dt.month.value_counts().sort_index().plot(kind='bar') 

### Section III - Q5: Make a bar plot showing the years in which movies with Ian McKellen tend to be released in the USA?

In [ ]:
mckellen_role_filter = (cast ['name'] == 'Ian McKellen') & (cast['n'] == 1.0) 
mckellen_over_years = cast[mckellen_role_filter].merge(release_dates[release_dates['country'] == 'USA'], on=['title', 'year']).sort_values('date', ascending=False) 
mckellen_over_years.date.dt.month.value_counts().sort_index().plot(kind='bar') 
